In [1]:
import pdblp
from xbbg import blp
import workdays
import time
import datetime
import pandas as pd
pd.set_option('display.max_columns', 70)

con = pdblp.BCon(timeout=10000)
con.start()

L = ["USD","EUR"]
# L = ["USD","EUR","GBP","CAD","AUD","NZD","CHF","NOK","SEK","HKD","CNY"]

T = ["CSBS" + i + " TMUQ Curncy" for i in L]  #Mitsubishi TTM
# T = [i  + "JPY CMPN Curncy" for i in L] #CMPN

tmp = ["px_last","volatility_90d","rsi_30d"]
BDP = blp.bdp(tickers=T, flds=tmp)
BDP = BDP.loc[T,:]

for i in tmp:
    BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
BDP.insert(0, "Currency", L )
BDP.insert(1, "Ticker", T)
BDP = BDP.reset_index(drop=True)
BDP = BDP.rename(columns={"volatility_90d":"vola"})

BDP

,Currency,Ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,139.15,11.32,56.49
1,EUR,CSBSEUR TMUQ Curncy,150.00,10.89,56.29


In [2]:
d_from = "20050301"
d_to = "20211027"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

TTM = con.bdh(T, "px_open", d_from, d_to,
             elms = [("nonTradingDayFillOption","NON_TRADING_WEEKDAYS"),("nonTradingDayFillMethod","PREVIOUS_VALUE")]).reset_index()
TTM = TTM[["date"]+T]
TTM.columns = ["Date"] + L
TTM.head()

,Date,USD,EUR
0,2005-03-01,104.52,138.11
1,2005-03-02,104.64,137.75
2,2005-03-03,104.99,137.83
3,2005-03-04,105.38,138.12
4,2005-03-07,104.72,138.65


In [3]:
LSP = []
T2 = []
LS = L + ["JPY"]
for i in range(len(LS)):
    for j in range(i+1,len(LS)):
        tmp = LS[i] + LS[j]
        LSP += [ tmp ]
        T2 += [ tmp + " CMPN Curncy" ]

CMPN = con.bdh(T2, "px_last", d_from, d_to,
             elms = [("nonTradingDayFillOption","NON_TRADING_WEEKDAYS"),("nonTradingDayFillMethod","PREVIOUS_VALUE")]).reset_index()
CMPN = CMPN[["date"]+T2]
CMPN.head()
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

ticker,date,USDEUR CMPN Curncy,USDJPY CMPN Curncy,EURJPY CMPN Curncy
field,,px_last,px_last,px_last
0,2005-03-01,0.7583,104.39,137.66
1,2005-03-02,0.7611,104.75,137.65
2,2005-03-03,0.7628,105.29,138.02
3,2005-03-04,0.7554,104.78,138.70
4,2005-03-07,0.7568,105.19,138.99
